In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.decomposition import PCA
# from sklearn.preprocessing import Imputer
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn.metrics import make_scorer
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
# import tflearn
# import tensorflow as tf
import seaborn
import warnings
warnings.filterwarnings('ignore')

from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

In [29]:
train = pd.read_csv('train_t0.csv')
label = pd.read_csv('train_t1.csv')

# test = pd.read_csv('tessv')
data = pd.concat([train],ignore_index=True)
data = data.drop("ID",1)
labels = label.drop("ID",1)


In [22]:
test = data[-15:]
test_labels = labels[-15:]
train = data[:-15]
train_labels = labels[:-15]
scaler = StandardScaler()


# Fit on training set only.
scaler.fit(train)
# Apply transform to both the training set and the test set.
train = scaler.transform(train)
test = scaler.transform(test)


In [23]:
pca = PCA(0.70)
pca.fit(train)
train = pca.transform(train)
test = pca.transform(test)
train.shape

(135, 16)

In [24]:
# R2 Score
from sklearn.multioutput import MultiOutputRegressor

def lets_try(train,train_labels,test,test_labels):
    results={}
    
    print(train.shape)
    print(test.shape)
    print(train_labels.shape)
    print(test_labels.shape)
    def test_model(clf):
        
        # cv = KFold(n_splits=5,shuffle=True,random_state=45)
        # r2 = make_scorer(mean_squared_error)
        # r2_val_score = cross_val_score(clf, train, labels, cv=cv,scoring=r2)
        # scores=[r2_val_score.mean()]
        
        clf.fit(train, train_labels)
        test_predict = clf.predict(test)
        scores = mean_squared_error(test_labels, test_predict)

        return [scores]

    clf = linear_model.LinearRegression()
    print(f"Linear Regression = {test_model(clf)}")
    
    clf = linear_model.Ridge()
    print(f"Ridge = {test_model(clf)}")
    
    # clf = linear_model.BayesianRidge()
    # print(f"BayesianRidge = {test_model(clf)}")
    
    # clf = linear_model.HuberRegressor()
    # print(f"HuberRegressor= {test_model(clf)}")
    
    clf = linear_model.Lasso(alpha=1e-4)
    print(f"Lasso = {test_model(clf)}")
    
    clf = BaggingRegressor()
    print(f"BaggingRegressor = {test_model(clf)}")
    
    clf = RandomForestRegressor()
    print(f"RandomForestRegressor= {test_model(clf)}")
    
    # clf = MultiOutputRegressor(AdaBoostRegressor())
    # print(f"AdaBoostRegressor = {test_model(clf)}")
    
    clf = MultiOutputRegressor(svm.SVR())
    print(f"SVM RBF = {test_model(clf)}")
    
    clf = MultiOutputRegressor(svm.SVR(kernel="linear"))
    print(f"SVM Linear = {test_model(clf)}")
    
    # results = pd.DataFrame.from_dict(results,orient='index')
    # results.columns=["MSE"] 
    # results=results.sort(columns=["MSE"],ascending=False)
    # results.plot(kind="bar",title="Model Scores")
    # axes = plt.gca()
    # axes.set_ylim([0.5,1])
    # return results

# lets_try(train,train_labels)

lets_try(train,train_labels,test,test_labels)

(135, 16)
(15, 16)
(135, 595)
(15, 595)
Linear Regression = [0.002596325787652626]
Ridge = [0.0025961320992643933]
Lasso = [0.0025950992079001494]
BaggingRegressor = [0.0026501670722064823]
RandomForestRegressor= [0.0024977426619862365]
SVM RBF = [0.00426763734108063]
SVM Linear = [0.004026502372019386]


In [34]:
test = pd.read_csv('test_t0.csv')
test = test.drop("ID",1)

In [35]:
train = data
train_labels = labels
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(train)
# Apply transform to both the training set and the test set.
train = scaler.transform(train)
test = scaler.transform(test)

pca = PCA(0.70)
pca.fit(train)
train = pca.transform(train)
test = pca.transform(test)

In [38]:
clf = RandomForestRegressor()
clf.fit(train, train_labels)
test_predict = clf.predict(test)
prediction = clf.predict(test)

In [39]:
print(prediction.shape)
df = pd.DataFrame(prediction)
df.head()

(80, 595)


,0,1,2,3,4,5,6,7,8,9,...,585,586,587,588,589,590,591,592,593,594
0,0.052133,0.089064,0.109771,0.0,0.052133,0.089064,0.127915,0.151494,0.082125,0.127915,...,0.131891,0.116678,0.195619,0.160934,0.144621,0.133619,0.140099,0.037947,0.037947,0.262275
1,0.043252,0.069146,0.067696,0.0,0.043252,0.069146,0.117431,0.108005,0.053658,0.117431,...,0.101268,0.105481,0.152376,0.120882,0.110440,0.107156,0.098212,0.023636,0.023636,0.203370
2,0.055525,0.075806,0.082976,0.0,0.055525,0.075806,0.133955,0.144963,0.081123,0.133955,...,0.126538,0.136963,0.194228,0.160383,0.144103,0.140645,0.141135,0.040776,0.040776,0.237087
3,0.041361,0.072274,0.058512,0.0,0.041361,0.072274,0.121925,0.099002,0.050154,0.121925,...,0.098788,0.080904,0.160684,0.120617,0.120944,0.111401,0.105711,0.025745,0.025745,0.252384
4,0.060524,0.077930,0.060357,0.0,0.060524,0.077930,0.132974,0.093639,0.056267,0.132974,...,0.098412,0.080831,0.161618,0.122389,0.110723,0.111413,0.101284,0.031969,0.031969,0.257160


In [40]:
meltedDF = df.to_numpy().flatten()
print(meltedDF.shape)

(47600,)


In [41]:
anan = pd.DataFrame(meltedDF)
anan.head()

,0
0,0.052133
1,0.089064
2,0.109771
3,0.000000
4,0.052133


In [162]:
anan.head()

,0
0,0.061191
1,0.069773
2,0.100056
3,0.000000
4,0.061191


In [163]:
anan.tail()

,0
47595,0.134184
47596,0.134456
47597,0.037313
47598,0.037313
47599,0.310460


In [164]:
print(anan.shape)

(47600, 1)


In [50]:
baban = anan.rename(columns={0: "predicted"})

In [47]:
baban = baban.set_index('predicted')

In [51]:
csvdosyasiseysi = baban.to_csv("test_results4.csv")

In [49]:
baban.head(
)

,level_0,index
predicted,,
0.052133,0,0
0.089064,1,1
0.109771,2,2
0.000000,3,3
0.052133,4,4
